In [1]:
import torch
from torch import nn
from torch.utils.data import Dataset, DataLoader
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from tqdm import tqdm

In [2]:
class SphereDatasetAnchor(Dataset):
    def __init__(self, extra_modes, num_points, transform_fn=None):
        self.n = num_points
        self.m = extra_modes + 1
        self.points = np.random.uniform(-1,1,size=(self.n, 3))
        mags = np.linalg.norm(self.points, axis=-1)
        self.points /= mags[...,None]
        self.disks = np.array([np.random.uniform(-1,1,size=(3)) for i in range(extra_modes)])
        self.disks = np.array([disk / np.linalg.norm(disk) for disk in self.disks])
        self.disk_points = torch.tensor(np.array([self.project_points(self.points, disk) for disk in self.disks])).float()
        self.points = torch.tensor(self.points).float()
        self.disks = torch.tensor(self.disks).float()
        self.transform_fn = transform_fn
        self.all_points = [self.points] + list(self.disk_points)
        
    def project_points(self, points, disk):
        output_points = []
        for point in points:
            cosval = np.dot(point, disk)
            arrow = (point*1/np.dot(point, disk) - disk) * cosval # mind the sign of cosval
            arrow /= np.linalg.norm(arrow)
            output_points.append(arrow)
        return np.array(output_points)
    
    def __len__(self):
        return 1
    
    def __getitem__(self, idx):
        if self.transform_fn is not None:
            return [self.transform_fn(point) for point in self.all_points]
        return self.all_points

In [3]:
from torch.nn.functional import cross_entropy, relu
class Mapper(nn.Module):
    def __init__(self):
        super().__init__()
        self.linear = nn.Linear(3, 128)
        self.linear2 = nn.Linear(128, 128)
        self.linear3 = nn.Linear(128, 128)
    
    def forward(self, X):
        return self.linear3(relu(self.linear2(relu(self.linear(X)))))

    def evaluate(self, X):
        self.eval()
        with torch.no_grad():
            return self.forward(X)

mapper = Mapper()

In [4]:
modes = 3
dataset = SphereDatasetAnchor(modes, 300, transform_fn=mapper)

In [18]:
from torch.nn.functional import cross_entropy, relu
class ContrastiveModel(nn.Module):
    def __init__(self, modes):
        super().__init__()
        self.linears = nn.ModuleList([nn.Linear(128, 128) for i in range(modes)])
        self.linears2 = nn.ModuleList([nn.Linear(128, 128) for i in range(modes)])
        self.linears3 = nn.ModuleList([nn.Linear(128, 64) for i in range(modes)])
        self.linears4 = nn.ModuleList([nn.Linear(64, 3) for i in range(modes)])
        self.comps = nn.ParameterList([nn.Parameter(torch.zeros(3,3)) for i in range(modes-1)])
        for comp in self.comps:
            nn.init.xavier_uniform_(comp)
    
    def loss(self, corr_mat):
        losses_row = cross_entropy(corr_mat, torch.arange(0, len(corr_mat)).long())
        losses_col = cross_entropy(torch.transpose(corr_mat,0,1), torch.arange(0, len(corr_mat)).long())
        return torch.mean(losses_row) + torch.mean(losses_col)
    
    def compute_corr_dot(self, a_points, b_points):
        return torch.sum(a_points[:,None] * b_points[None,:], dim=-1)
    
    def compute_corr_func(self, a_points, b_points, b):
        return self.comps4[b-1](relu(self.comps3[b-1](relu(self.comps2[b-1](relu(self.comps[b-1](a_points[:,None] * b_points[None,:])))))))[...,0]
        #return self.compute_corr_dot(a_points_mapped, b_points)
    
    def forward(self, a, b, a_points, b_points):
        a_points = self.run_evaluate(a, a_points)
        b_points = self.run_evaluate(b, b_points)
        
        # a_points = torch.tensordot(a_points, (self.comps[b-1] + self.comps[b-1].T)/2, dims=1)
        # b_points = torch.tensordot(b_points, (self.comps[b-1] + self.comps[b-1].T)/2, dims=1)
        # a_points = a_points / torch.linalg.norm(a_points, dim=-1, keepdim=True)
        # b_points = b_points / torch.linalg.norm(b_points, dim=-1, keepdim=True)
        corr_mat = self.compute_corr_dot(a_points, b_points)
        return corr_mat
    
    def run_evaluate(self, i, points):
        #points = self.linears2[i](relu(self.linears[i](points)))
        points = self.linears4[i](relu(self.linears3[i](relu(self.linears2[i](relu(self.linears[i](points)))))))
        points = points / torch.linalg.norm(points, dim=-1, keepdim=True)
        return points
       

In [19]:
model = ContrastiveModel(modes+1)

In [20]:
from torch.optim import Adam
dataloader = DataLoader(dataset, batch_size=1, shuffle=True)
optim = Adam(model.parameters(), lr=0.00003)

In [21]:
def train_data(optim, model, all_points):
    ptrs1 = all_points[0]
    total_loss = 0
    for b, ptrs2 in enumerate(all_points[1:]):
        optim.zero_grad()
        corr_mat = model(0, b+1, ptrs1, ptrs2)
        loss = model.loss(corr_mat)
        total_loss += loss
    total_loss /= len(all_points[1:])
    total_loss.backward()
    optim.step()
    return total_loss.item()

In [22]:
EPOCH = 2000
progress_bar = tqdm(range(EPOCH), desc='Epoch loss') 
for epoch in range(EPOCH):
    all_points = dataset[0]
    loss_val = train_data(optim, model, all_points)
    progress_bar.set_postfix({'loss': loss_val})

Epoch loss:   0%|          | 0/2000 [03:47<?, ?it/s, loss=9.93]


In [23]:
orig_points = dataset.all_points

In [24]:
import plotly.express as px
import pandas as pd

In [25]:
def plot_points(points):
    df = pd.DataFrame(np.concatenate(points, axis=0), columns=['x', 'y', 'z'])
    df['type'] = sum([[f'{i}']*300 for i in range(len(points))], [])
    fig = px.scatter_3d(df, x='x', y='y', z='z', color='type', opacity=1)
    fig.update_traces(marker={'size': 5})
    fig.show()

In [26]:
plot_points(orig_points)

In [27]:
pred_points = [model.run_evaluate(i, mapper(points)).detach().numpy() for i, points in enumerate(orig_points)]

In [28]:
plot_points(pred_points)

In [29]:
model.loss(model.compute_corr_dot(orig_points[1], orig_points[1]))

tensor(9.8856)

In [142]:
a = (model.comps[0]+model.comps[0].T)/2
print(a)
torch.linalg.eig(a)

tensor([[-0.2518,  0.3999, -0.5836],
        [ 0.3999,  0.9914, -0.8119],
        [-0.5836, -0.8119, -0.9017]], grad_fn=<DivBackward0>)


torch.return_types.linalg_eig(
eigenvalues=tensor([ 1.4905+0.j, -0.2823+0.j, -1.3703+0.j], grad_fn=<LinalgEigBackward0>),
eigenvectors=tensor([[ 0.3247+0.j,  0.8665+0.j,  0.3791+0.j],
        [ 0.8687+0.j, -0.4317+0.j,  0.2428+0.j],
        [-0.3741+0.j, -0.2505+0.j,  0.8929+0.j]], grad_fn=<LinalgEigBackward0>))

In [165]:
b=1
points = torch.tensor(pred_points[0])
#points = torch.tensordot(points, (model.comps[b-1] + model.comps[b-1].T)/2, dims=1)
#points /= torch.linalg.norm(points, dim=-1, keepdim=True)
new_points = [ptrs.copy() for ptrs in pred_points]
new_points[0] = points.detach()
df = pd.DataFrame(np.concatenate(new_points, axis=0), columns=['x', 'y', 'z'])
df['type'] = sum([[f'{i}']*300 for i in range(modes+1)], [])
fig = px.scatter_3d(df, x='x', y='y', z='z', color='type')
fig.update_traces(marker={'size': 5})
fig.show()

In [56]:
def comp_stable_rank(ptrs):
    corr = np.matmul(ptrs.T, ptrs)/len(ptrs)
    sing = np.linalg.svd(corr)[1][0]
    norm = np.linalg.norm(corr)
    return norm**2/sing**2
    
print(comp_stable_rank(pred_points[0]))
print(comp_stable_rank(pred_points[1]))

1.0441717074138692
1.594033906400954


In [57]:
ptrs1 = dataset.points
ptrs2 = dataset.disk_points[1]
loss_total = 0
for i,ptr2 in enumerate(ptrs2):
    loss_total += cross_entropy(torch.sum(ptrs1 * ptr2[None,:], dim=-1), torch.tensor(i).long())
for i,ptr1 in enumerate(ptrs1):
    loss_total += cross_entropy(torch.sum(ptr1[None,:] * ptrs2, dim=-1), torch.tensor(i).long())
print(loss_total/len(ptrs2))

tensor(10.1642)


In [58]:
l1 = cross_entropy(torch.sum(ptrs1[:,None]*ptrs2[None,:], dim=-1), torch.arange(len(ptrs1)).long())
l2 = cross_entropy(torch.sum(ptrs2[:,None]*ptrs1[None,:], dim=-1), torch.arange(len(ptrs1)).long())
l1+l2

tensor(10.1642)

In [62]:
model.loss(model.compute_corr_dot(ptrs2, ptrs2))

tensor(12.2896)